# 08 - Comparación con AVA y Métricas Clínicas

## Objetivo
Comparar outputs con AVA 3.2 o análisis manual:
- MFI, PPV, PVD
- Coherencia hemodinámica


In [ ]:
import seaborn as sns
import pandas as pd

df = pd.read_csv("results_comparison.csv")
sns.scatterplot(data=df, x="PVD_AVA", y="PVD_model")
plt.plot([0, df.max().max()], [0, df.max().max()], 'r--')
plt.title("Comparación de PVD")
